In [ ]:
import pandas as pdd

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [12]:
df=pd.read_csv('train.csv') # df는 트레인용

In [13]:
# df # label =0, 1(악성)
# 6,995,056 rows * 3 columns

In [14]:
# URL 텍스트를 벡터화
vectorizer = CountVectorizer() # 텍스트에 등장하는 단어의 빈도수를 세어, 벡터형태로 변환
X = vectorizer.fit_transform(df['URL'])
y = df['label']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [16]:
'''
# alpha 튜닝 (recall 기준)
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10]
}

grid_model = GridSearchCV(
    MultinomialNB(),
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_model.fit(X_train, y_train)

# 5. 최적 모델 정보 출력
print(f"\nBest alpha: {grid_model.best_params_['alpha']}")
print(f"Best cross-validated recall: {grid_model.best_score_:.4f}")

# 6. 검증셋에서 성능 평가
best_model = grid_model.best_estimator_
y_pred = best_model.predict(X_test)

print("\n[검증 셋 평가 결과]")
print(classification_report(y_test, y_pred))
# alpha=0.01이 best
'''

'\n# alpha 튜닝 (recall 기준)\nparam_grid = {\n    \'alpha\': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10]\n}\n\ngrid_model = GridSearchCV(\n    MultinomialNB(),\n    param_grid=param_grid,\n    scoring=\'recall\',\n    cv=5,\n    n_jobs=-1,\n    verbose=1\n)\n\ngrid_model.fit(X_train, y_train)\n\n# 5. 최적 모델 정보 출력\nprint(f"\nBest alpha: {grid_model.best_params_[\'alpha\']}")\nprint(f"Best cross-validated recall: {grid_model.best_score_:.4f}")\n\n# 6. 검증셋에서 성능 평가\nbest_model = grid_model.best_estimator_\ny_pred = best_model.predict(X_test)\n\nprint("\n[검증 셋 평가 결과]")\nprint(classification_report(y_test, y_pred))\n# alpha=0.01이 best\n'

In [17]:
# 나이브베이즈 모델로 지도학습
model = MultinomialNB(alpha=0.01)
model.fit(X_train, y_train)

# 교차 검증 (CV: Cross Validation)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"교차 검증 정확도 평균: {scores.mean():.4f}")

교차 검증 정확도 평균: 0.9570


In [21]:
# model fit 후에 평가
y_pred = model.predict(X_test)

print("\n[테스트 셋 평가 결과]")
print(classification_report(y_test, y_pred))


[테스트 셋 평가 결과]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97    543432
           1       0.92      0.89      0.91    156074

    accuracy                           0.96    699506
   macro avg       0.94      0.93      0.94    699506
weighted avg       0.96      0.96      0.96    699506



In [19]:
test_df = pd.read_csv("test.csv")

In [20]:
# 벡터화 (훈련에 사용한 vectorizer로 변환)
tdf = vectorizer.transform(test_df['URL'])

# 확률 구하기
y_proba = model.predict_proba(tdf)[:, 1] # label이 1인 확률만 저장

# submission에 뭐 넣을지
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'probability': y_proba
})

# CSV로 저장
# sample_submission.csv 제출용
submission.to_csv("submission.csv", index=False)
print(">>> submission.csv 저장 완료 ✅")

>>> submission.csv 저장 완료 ✅
